<a href="https://colab.research.google.com/github/DanhChun/Log-Analyzer-with-NLP-model/blob/main/Distilation_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LOAD DỮ LIỆU VÀ KIỂM TRA DỮ LIỆU**

In [ ]:
from google.colab import files
import pandas as pd

# Tải lên file từ máy tính
uploaded = files.upload()

Saving labeled_logs.csv to labeled_logs.csv


In [ ]:
# Đọc dữ liệu từ file CSV

df = pd.read_csv("labeled_logs.csv")

# Hiển thị một vài dòng đầu tiên
df.head()

,logs,labels
0,00:00:13.122..\src\mgw3\mgwp.py.282.INFO: -> d...,Normal
1,00:00:13.123..\src\mgw3\mgwp.py.50.INFO: -> ap...,Normal
2,00:00:13.124..\src\mgw3\mgwp.py.349.INFO:app_c...,Normal
3,00:00:13.125..\src\mgw3\sif\sif_mgr.py.68.INFO...,Normal
4,00:00:13.125..\src\mgw3\sif\sif_mgr.py.70.INFO...,Normal


**LÀM SẠCH DỮ LIỆU**

In [ ]:
import pandas as pd
import re

# Đọc dữ liệu
df = pd.read_csv("labeled_logs.csv")

# Chuyển logs thành chữ thường
df["logs"] = df["logs"].str.lower()

# Làm sạch nhãn (labels)
df["labels"] = df["labels"].astype(str).str.strip()

# Xóa dòng có giá trị rỗng ở bất kỳ cột nào
df.dropna(subset=["logs", "labels"], inplace=True)

# Hàm làm sạch logs
def clean_log_text(text):
    # Xoá các ký tự đặc biệt cụ thể: " \ ( ) { } [ ] ! #
    cleaned = re.sub(r'[\"\\()\[\]{}!#]', '', text)
    # Xoá khoảng trắng dư thừa
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    return cleaned

# Áp dụng hàm làm sạch cho cột logs
df["logs"] = df["logs"].apply(clean_log_text)

# Kiểm tra lại vài dòng đầu
print(df.head())


                                                logs  labels
0  00:00:13.122..srcmgw3mgwp.py.282.info: -> do r...  Normal
1  00:00:13.123..srcmgw3mgwp.py.50.info: -> app_e...  Normal
2  00:00:13.124..srcmgw3mgwp.py.349.info:app_clos...  Normal
3  00:00:13.125..srcmgw3sifsif_mgr.py.68.info: ->...  Normal
4  00:00:13.125..srcmgw3sifsif_mgr.py.70.info: ->...  Normal


**TẠO WORD MAP CHO CÁC BIẾN THỂ DỮ LIỆU**

In [ ]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import pandas as pd
import random

# Đọc dữ liệu logs
df = pd.read_csv("labeled_logs.csv")

#  Với lỗi chứa từ khóa có thể thay thế (thay thế từ đồng nghĩa)
synonym_map = {
    "problem": ["issue", "fault", "malfunction"],
    "error": ["failure", "fault", "bug"],
    "failed": ["crashed", "unsuccessful", "did not succeed"],
    "download": ["get", "acquire", "http_download", "missing","fetch data", "data retrieval"],
    "http_download error": ["notice acquire failure", "message get fault", "data get bug"],
    "disconnect": ["lost connection", "disconnection", "drop"],
    "checksum": ["CRC", "hash check", "data integrity check"],
    "message size": ["packet size", "data length", "buffer size", "size"]
}

#  Với lỗi chứa thông số kỹ thuật hoặc tên thiết bị
device_map = {
    "control": ["amplifier control", "control_amplifier", "control_ampli", "control amplifier", "modify volume", ],
    "SIM": ["subscriber identity module", "mobile SIM", "network SIM"],
    "GSM": ["cellular network", "mobile network", "telecom system"],
    "config": ["construct", "prepare"],
    "config SIM, GSM": ["construct mobile SIM, mobile network", "prepare network SIM, telecom system", "config subscriber identity module, cellular network"]
}

#  Với các thông báo mang tính mô tả
phrase_map = {
    "message played done": ["audio playback finished", "sound message completed", "-> playing changed: ON -> OFF", "ON -> OFF", "playing changed", "message played OFF"]
}

#  Với nhãn đặc biệt như "Normal"
normal_variations = [
    "System stable",
    "No issues detected",
]

# Hàm thay thế từ đồng nghĩa và thuật ngữ liên quan
def augment_text(text):
    original_text = text  # Lưu văn bản gốc để so sánh sau

    # Thay thế từ đồng nghĩa
    for phrase, synonyms in synonym_map.items():
        if phrase in text:
            synonym = random.choice(synonyms)
            text = text.replace(phrase, synonym)

    # Thay thế thuật ngữ thiết bị
    for phrase, variations in device_map.items():
        if phrase in text:
            variation = random.choice(variations)
            text = text.replace(phrase, variation)

    # Thay đổi cách diễn đạt của mô tả lỗi
    for phrase, variations in phrase_map.items():
        if phrase in text:
            variation = random.choice(variations)
            text = text.replace(phrase, variation)

    # Nếu nhãn là "Normal", thay thế ngẫu nhiên bằng cách diễn đạt khác
    if text.strip().lower() == "normal":
        text = random.choice(normal_variations)

    # Kiểm tra nếu có thay đổi
    if original_text != text:
        print(f"Original: {original_text}")
        print(f"Augmented: {text}")

    return text

# Áp dụng augmentation cho toàn bộ tập dữ liệu
df["augmented_logs"] = df["logs"].apply(augment_text)

# Kiểm tra dữ liệu đã augmented
print(df.head())  # In ra vài dòng đầu tiên của dữ liệu đã thay đổi

# Lưu kết quả
df.to_csv("augmented_labeled_logs.csv", index=False)

Original: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli error
Augmented: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: amplifier control_ampli bug
Original: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli error
Augmented: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli_ampli failure
Original: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli error
Augmented: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_amplifier_ampli failure
Original: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli error
Augmented: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: amplifier control_ampli failure
Original: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli error
Augmented: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: modify volume_ampli bug
Original: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli error
Augmented: 00:00:46.938..\src\mgw3\htttn\htttn_rx

In [ ]:
# Kiểm tra nếu có sự thay đổi giữa logs và augmented_logs
for i, row in df.iterrows():
    if row['logs'] != row['augmented_logs']:
        print(f"Original: {row['logs']}")
        print(f"Augmented: {row['augmented_logs']}")
        print('-' * 50)


Original: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli error
Augmented: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: amplifier control_ampli bug
--------------------------------------------------
Original: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli error
Augmented: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli_ampli failure
--------------------------------------------------
Original: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli error
Augmented: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_amplifier_ampli failure
--------------------------------------------------
Original: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli error
Augmented: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: amplifier control_ampli failure
--------------------------------------------------
Original: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli error
Augmented: 00:

**TOKEN HOÁ VÀ MAPPING NHÃN**

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Đọc dữ liệu từ tệp CSV
df = pd.read_csv('augmented_labeled_logs.csv')

# Mã hóa nhãn thành số
label_encoder = LabelEncoder()
df['encoded_labels'] = label_encoder.fit_transform(df['labels'])

# Lưu lại dữ liệu với nhãn đã mã hóa
df.to_pickle('encoded_labeled_logs.pkl')  # Lưu dưới dạng pickle
df.to_csv('encoded_labeled_logs.csv', index=False)  # Lưu dưới dạng CSV

# Xem trước một số mẫu nhãn đã mã hóa
print(df[['labels', 'encoded_labels']].head())


   labels  encoded_labels
0  Normal               8
1  Normal               8
2  Normal               8
3  Normal               8
4  Normal               8


In [ ]:
# Liệt kê các nhãn gốc và nhãn mã hóa tương ứng
encoded_labels_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
print("Nhãn gốc và nhãn mã hóa:")
for label, encoded in encoded_labels_mapping.items():
    print(f"{label}: {encoded}")


Nhãn gốc và nhãn mã hóa:
ERROR - MPD problem: 0
ERROR - config SIM, GSM: 1
ERROR - control amplifier: 2
ERROR - delete message: 3
ERROR - invalid checksum: 4
ERROR - invalid message size: 5
ERROR - message download: 6
ERROR - start MPD failed: 7
Normal: 8
Note - message played done: 9
WARNING - On disconnect: 10


In [ ]:
df = df.dropna()


In [ ]:
import torch
from transformers import DistilBertTokenizer

# Đọc dữ liệu đã mã hóa nhãn từ tệp CSV
df = pd.read_csv('encoded_labeled_logs.csv')

# Tải tokenizer của DistilBERT từ Hugging Face
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Chuyển các cột pandas Series thành list
train_texts = df['augmented_logs'][:int(0.8 * len(df))].astype(str).tolist()
val_texts = df['augmented_logs'][int(0.8 * len(df)):].astype(str).tolist()

# Tokenize train và validation set
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

# Chuyển đổi các encoding thành tensor
train_input_ids = torch.tensor(train_encodings['input_ids'])
train_attention_mask = torch.tensor(train_encodings['attention_mask'])
val_input_ids = torch.tensor(val_encodings['input_ids'])
val_attention_mask = torch.tensor(val_encodings['attention_mask'])


In [ ]:
# In ra 5 câu đã được tokenized và input ids tương ứng
print("Sample tokenized text:")
for i in range(5):
    print(f"Original text: {train_texts[i]}")
    print(f"Tokenized ids: {train_encodings['input_ids'][i]}")
    print(f"Attention mask: {train_encodings['attention_mask'][i]}")
    print("-" * 40)


Sample tokenized text:
Original text: 00:00:13.122..\src\mgw3\mgwp.py.282.INFO: -> do RESTART APP at night!
Tokenized ids: [101, 4002, 1024, 4002, 1024, 2410, 1012, 13092, 1012, 1012, 1032, 5034, 2278, 1032, 11460, 2860, 2509, 1032, 11460, 2860, 2361, 1012, 1052, 2100, 1012, 26267, 1012, 18558, 1024, 1011, 1028, 2079, 23818, 10439, 2012, 2305, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Attention mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
inputs = tokenizer("13:40:57.691..\src\mgw3\htttn\htttn_mgr.py.357.INFO:_on_disconnect: client=<paho.mqtt.client.Client object at 0xb57c4c58>, rc=0", return_tensors="pt", truncation=True, padding=True, max_length=128)
print("Input IDs:", inputs["input_ids"])
print("Attention Mask:", inputs["attention_mask"])


Input IDs: tensor([[  101,  2410,  1024,  2871,  1024,  5401,  1012,  6353,  2487,  1012,
          1012,  1032,  5034,  2278,  1032, 11460,  2860,  2509,  1032,  1044,
          4779,  2102,  2078,  1032,  1044,  4779,  2102,  2078,  1035, 11460,
          2099,  1012,  1052,  2100,  1012, 26231,  1012, 18558,  1024,  1035,
          2006,  1035, 12532, 10087,  6593,  1024,  7396,  1027,  1026,  6643,
          6806,  1012,  1049,  4160,  4779,  1012,  7396,  1012,  7396,  4874,
          2012,  1014,  2595,  2497, 28311,  2278,  2549,  2278, 27814,  1028,
          1010, 22110,  1027,  1014,   102]])
Attention Mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]])


In [ ]:
print(tokenizer.convert_ids_to_tokens(5401))

57


**TẠO DATASET**

In [ ]:
from torch.utils.data import Dataset, DataLoader

# Mã hóa nhãn thành tensor
train_labels = torch.tensor(df['encoded_labels'][:int(0.8 * len(df))].tolist())
val_labels = torch.tensor(df['encoded_labels'][int(0.8 * len(df)):].tolist())

# Tạo Dataset cho PyTorch
class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

# Tạo dataset cho train và validation
train_dataset = CustomDataset(train_input_ids, train_attention_mask, train_labels)
val_dataset = CustomDataset(val_input_ids, val_attention_mask, val_labels)

# Tạo DataLoader cho train và validation
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16)

# Lưu lại tệp dữ liệu tokenized
df['input_ids'] = list(train_input_ids.numpy()) + list(val_input_ids.numpy())
df['attention_mask'] = list(train_attention_mask.numpy()) + list(val_attention_mask.numpy())
df['encoded_labels'] = list(train_labels.numpy()) + list(val_labels.numpy())

# Lưu dưới dạng pickle và CSV
df.to_pickle('tokenized_encoded_labeled_logs.pkl')  # Lưu dưới dạng pickle
df.to_csv('tokenized_encoded_labeled_logs.csv', index=False)  # Lưu dưới dạng CSV


**CẤU HÌNH TRAIN**

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

num_labels = len(set(df["labels"]))  # Số lượng nhãn

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**CẤU HÌNH HÀM ĐÁNH GIÁ MÔ HÌNH**

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Hàm tính độ chính xác
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Chọn nhãn có xác suất cao nhất
    acc = accuracy_score(labels, predictions)  # Tính độ chính xác
    return {"accuracy": acc}

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Chọn nhãn có xác suất cao nhất
    acc = accuracy_score(labels, predictions)  # Tính độ chính xác
    precision = precision_score(labels, predictions, average='weighted')  # Tính precision
    recall = recall_score(labels, predictions, average='weighted')  # Tính recall
    f1 = f1_score(labels, predictions, average='weighted')  # Tính F1-score
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

**THIẾT LẬP THÔNG SỐ TRAIN**

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    report_to="none",  # Tắt W&B
    #evaluation_strategy = "epoch",
    #save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,  # 🔹 Đảm bảo số epochs như mong muốn
    weight_decay=0.01,
    logging_dir="./logs",
)


In [ ]:
import torch

# Kiểm tra và sử dụng GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")  # Kiểm tra thiết bị đang sử dụng


Using device: cuda


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


**TIẾN HÀNH TRAIN**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import Trainer
from sklearn.metrics import balanced_accuracy_score
import numpy as np

# Hàm tính Balanced Accuracy
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "balanced_accuracy": balanced_accuracy_score(labels, preds)
    }

# Lớp Trainer chuẩn sử dụng Cross Entropy Loss
class SimpleTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")  # Lấy labels
        outputs = model(**inputs)  # Truyền inputs vào model
        logits = outputs.logits  # Lấy logits

        # Sử dụng Cross Entropy Loss mà không có class weights
        loss_fct = nn.CrossEntropyLoss()  # Không có trọng số lớp
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss

# Khởi tạo Trainer với Cross Entropy Loss và balanced accuracy
trainer = SimpleTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Step,Training Loss
500,0.333400


TrainOutput(global_step=570, training_loss=0.2968161654054073, metrics={'train_runtime': 126.8546, 'train_samples_per_second': 71.68, 'train_steps_per_second': 4.493, 'total_flos': 301179846272256.0, 'train_loss': 0.2968161654054073, 'epoch': 3.0})

**ĐÁNH GIÁ KẾT QUẢ MÔ HÌNH**

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


{'eval_loss': 0.1342516541481018,
 'eval_balanced_accuracy': 0.9960623513208964,
 'eval_runtime': 3.0077,
 'eval_samples_per_second': 252.023,
 'eval_steps_per_second': 15.959,
 'epoch': 3.0}

**LƯU MÔ HÌNH Ở DẠNG FILE CỦA TRANSFORMERS**

In [ ]:
model.save_pretrained("./finetuned_distilber")
tokenizer.save_pretrained("./finetuned_distilber")

('./finetuned_distilber/tokenizer_config.json',
 './finetuned_distilber/special_tokens_map.json',
 './finetuned_distilber/vocab.txt',
 './finetuned_distilber/added_tokens.json')

**KIỂM TRA HOẠT ĐỘNG CỦA MÔ HÌNH FINE-TUNE**

In [ ]:
import torch

# Lấy thiết bị của mô hình (CPU hoặc CUDA)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Đưa mô hình về đúng thiết bị

def predict_log(log_text):
    inputs = tokenizer(log_text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Chuyển input lên cùng thiết bị với model

    with torch.no_grad():
        outputs = model(**inputs)

    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    return label_encoder.inverse_transform([predicted_class])[0]

# Ví dụ dự đoán
log_sample = "Error: invalid checksum"
print(predict_log(log_sample))


ERROR - invalid checksum


In [ ]:
def predict_log_proba(log_text):
    inputs = tokenizer(log_text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()

    label = label_encoder.inverse_transform([predicted_class])[0]
    confidence = probabilities[0][predicted_class].item()

    return label, confidence

# Ví dụ dự đoán
log_sample = "20:43:15.156..\src\mgw3\htttn\htttn_rx.py.75.INFO:invalid size"
predicted_label, confidence = predict_log_proba(log_sample)
print(f"Predicted: {predicted_label} (Confidence: {confidence:.2f})")


Predicted: ERROR - invalid message size (Confidence: 0.95)


In [ ]:
def predict_log_verbose(log_text):
    inputs = tokenizer(log_text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)  # Chuyển sang xác suất
    predicted_class = torch.argmax(logits, dim=1).item()

    print("Logits:", logits)
    print("Probabilities:", probabilities)
    print("Predicted class index:", predicted_class)
    print("Predicted label:", label_encoder.inverse_transform([predicted_class])[0])

    return label_encoder.inverse_transform([predicted_class])[0]

# Chạy thử
log_sample = "13:40:57.691..\src\mgw3\htttn\htttn_mgr.py.357.INFO:_on_disconnect: client=<paho.mqtt.client.Client object at 0xb57c4c58>, rc=0"
predict_log_verbose(log_sample)


Logits: tensor([[-0.7368, -1.1126, -0.5871, -0.9053, -0.6178, -1.2362, -0.5692, -1.2233,
         -0.9924, -1.1025, -0.3647, -1.8125,  5.6203]], device='cuda:0')
Probabilities: tensor([[1.7036e-03, 1.1699e-03, 1.9785e-03, 1.4394e-03, 1.9187e-03, 1.0339e-03,
         2.0142e-03, 1.0473e-03, 1.3193e-03, 1.1817e-03, 2.4713e-03, 5.8099e-04,
         9.8214e-01]], device='cuda:0')
Predicted class index: 12
Predicted label: WARNING - On disconnect


'WARNING - On disconnect'

In [ ]:
#import os

#file_path = "./finetuned_distilber"
#file_size = os.path.getsize(file_path) / (1024 * 1024)  # Chuyển sang MB
#print(f"Kích thước mô hình: {file_size:.2f} MB")

Kích thước mô hình: 0.00 MB


**TRÍCH XUẤT KIẾN THỨC**

In [ ]:
!pip install transformers datasets accelerate evaluate -q
!pip install git+https://github.com/huggingface/optimum.git -q  # Dùng cho distillation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml

In [ ]:
# Tải mô hình đã fine-tune từ Hugging Face
from transformers import AutoModelForSequenceClassification, AutoTokenizer

teacher_model = AutoModelForSequenceClassification.from_pretrained("./finetuned_distilber")
teacher_model.eval()  # freeze nếu không fine-tune lại

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher_model.to(device)

tokenizer = AutoTokenizer.from_pretrained("./finetuned_distilber")


In [ ]:
!pip install optimum[onnxruntime] -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.6 MB/s eta 0:00:00


**CẤU HÌNH HÀM MẤT MÁT TRÍCH XUẤT**

In [ ]:
import torch.nn.functional as F
from transformers import AutoModelForSequenceClassification

student_model = DistilBertForSequenceClassification.from_pretrained(
    "sentence-transformers/all-MiniLM-L12-v2", num_labels=num_labels
)

# Chuyển inputs tới cùng thiết bị với teacher model
inputs = {key: value.to(device) for key, value in inputs.items()}

# Chuyển student model tới thiết bị giống của teacher model
student_model.to(device)

outputs_student = student_model(**inputs)
with torch.no_grad():
    outputs_teacher = teacher_model(**inputs)

def distillation_loss(student_logits, teacher_logits, labels, T=2.0, alpha=0.5):
    loss_ce = F.cross_entropy(student_logits, labels)
    loss_kl = F.kl_div(
        input=F.log_softmax(student_logits / T, dim=-1),
        target=F.softmax(teacher_logits / T, dim=-1),
        reduction="batchmean"
    ) * (T * T)
    return alpha * loss_kl + (1. - alpha) * loss_ce

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L12-v2 and are newly initialized: ['classifier.bias', 'classifier.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin2.bias', 'distilbert.transformer.layer.0.ffn.lin2.weight', 'distilbert.transformer.layer.0.output_layer_norm.bias', 'distilbert.transformer.layer.0.output_layer_norm.weight', 'distilbert.transformer.la

**CẤU HÌNH TRAINER**

In [ ]:
from transformers import Trainer

class DistillationTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):  # Add **kwargs
        labels = inputs.get("labels")
        # Forward của student
        outputs_student = model(**inputs)
        student_logits = outputs_student.logits

        # Forward của teacher (không backprop)
        with torch.no_grad():
            teacher_outputs = teacher_model(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"]
            )
            teacher_logits = teacher_outputs.logits

        loss = distillation_loss(student_logits, teacher_logits, labels)

        return (loss, outputs_student) if return_outputs else loss

In [ ]:
# Tải và xử lý dữ liệu logs
import pandas as pd
import torch
from torch.utils.data import Dataset

# Đọc lại file pickle đã lưu
df = pd.read_pickle("tokenized_encoded_labeled_logs.pkl")

**CẤU HÌNH DATASET CHO TRÍCH XUẤT**

In [ ]:
split_index = int(0.8 * len(df))

train_input_ids = torch.tensor(df['input_ids'][:split_index].tolist())
train_attention_mask = torch.tensor(df['attention_mask'][:split_index].tolist())
train_labels = torch.tensor(df['encoded_labels'][:split_index].tolist())

val_input_ids = torch.tensor(df['input_ids'][split_index:].tolist())
val_attention_mask = torch.tensor(df['attention_mask'][split_index:].tolist())
val_labels = torch.tensor(df['encoded_labels'][split_index:].tolist())


<ipython-input-32-a7bf3dfb4df7>:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  train_input_ids = torch.tensor(df['input_ids'][:split_index].tolist())


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

train_dataset = CustomDataset(train_input_ids, train_attention_mask, train_labels)
val_dataset = CustomDataset(val_input_ids, val_attention_mask, val_labels)


In [ ]:
!pip install --upgrade transformers


**TRAIN TRÍCH XUẤT**

In [ ]:
from transformers import TrainingArguments
#from transformers import AutoModelForSequenceClassification

# Khởi tạo student model
#student_model = DistilBertForSequenceClassification.from_pretrained(
#    "sentence-transformers/all-MiniLM-L12-v2", num_labels=num_labels)

# Cấu hình training_args
training_args.output_dir = "./distilbert_student_model"  # Kết quả lưu trữ
training_args.evaluation_strategy = "epoch"  # Tính toán đánh giá sau mỗi epoch
training_args.learning_rate = 5e-5  # Learning rate
training_args.per_device_train_batch_size = 8  # Batch size cho training
training_args.per_device_eval_batch_size = 8  # Batch size cho evaluation
training_args.num_train_epochs = 3  # Số lượng epoch
training_args.weight_decay = 0.01  # Weight decay
training_args.logging_dir = "./logs"  # Thư mục lưu logs
training_args.logging_steps = 100


trainer = DistillationTrainer(
    model=student_model,  # Mô hình student cần distillation
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()


Step,Training Loss
100,2.773800
200,2.348800
300,1.551200
400,1.443900
500,1.128100
600,0.944000
700,0.696300
800,0.548000
900,0.428900
1000,0.269200


TrainOutput(global_step=1407, training_loss=0.9145658756920693, metrics={'train_runtime': 181.8716, 'train_samples_per_second': 61.89, 'train_steps_per_second': 7.736, 'total_flos': 307932944320512.0, 'train_loss': 0.9145658756920693, 'epoch': 3.0})

In [ ]:
student_model.save_pretrained("./student_model_distilled")
tokenizer.save_pretrained("./student_model_distilled")


('./student_model_distilled/tokenizer_config.json',
 './student_model_distilled/special_tokens_map.json',
 './student_model_distilled/vocab.txt',
 './student_model_distilled/added_tokens.json',
 './student_model_distilled/tokenizer.json')

In [ ]:
import os

def get_model_size(path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size / (1024 * 1024)  # MB

size_in_mb = get_model_size("./student_model_distilled")
print(f"Kích thước mô hình student: {size_in_mb:.2f} MB")


Kích thước mô hình student: 182.27 MB


In [ ]:
eval_result = trainer.evaluate()
print(eval_result)


{'eval_loss': 0.5964368581771851, 'eval_runtime': 6.2508, 'eval_samples_per_second': 150.221, 'eval_steps_per_second': 18.878, 'epoch': 3.0}


**KIỂM TRA MÔ HÌNH TRÍCH XUẤT**

In [ ]:
log_example = "14:12:39.627..\src\mgw3\player\player_mpc.py.145.IINFO: -> MPD got problem (state no change) -> replay."
from transformers import pipeline

# Load mô hình đã lưu
from transformers import AutoModelForSequenceClassification, AutoTokenizer

student_model = AutoModelForSequenceClassification.from_pretrained("./student_model_distilled")
tokenizer = AutoTokenizer.from_pretrained("./student_model_distilled")

# Tạo pipeline inference
clf_pipeline = pipeline("text-classification", model=student_model, tokenizer=tokenizer)

# Phán đoán log
result = clf_pipeline(log_example)
print(result)
id2label = student_model.config.id2label
label_name = id2label[int(result[0]['label'].split("_")[-1])]
print(f"Lỗi dự đoán: {label_name} (score: {result[0]['score']:.2f})")
from sklearn.metrics import classification_report

# Dự đoán toàn bộ val set
preds_output = trainer.predict(val_dataset)
pred_labels = preds_output.predictions.argmax(-1)


Device set to use cuda:0


[{'label': 'LABEL_0', 'score': 0.8894491195678711}]
Lỗi dự đoán: LABEL_0 (score: 0.89)


In [ ]:
# In báo cáo phân loại
print(classification_report(val_labels.numpy(), pred_labels))


**CHUYỂN ĐỔI ONNOX**

In [ ]:
!pip install transformers onnxruntime optimum

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from optimum.exporters.onnx import main_export

main_export(
    model_name_or_path="./student_model_distilled",
    output="./onnx_model",
    task="text-classification",
    opset=14 # Change opset version to 14 or higher
)

**CHUYỂN ĐỔI TENSORFLOW LITE**

In [ ]:
!pip uninstall tf2onnx

!python -m tf2onnx.convert --onnx-model ./onnx_model/model.onnx --output model_tf.onnx --opset 12

Found existing installation: tf2onnx 1.16.1
Uninstalling tf2onnx-1.16.1:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/tf2onnx-1.16.1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/tf2onnx/*
Proceed (Y/n)? y
  Successfully uninstalled tf2onnx-1.16.1
/usr/bin/python3: Error while finding module specification for 'tf2onnx.convert' (ModuleNotFoundError: No module named 'tf2onnx')


In [ ]:
!pip install onnx onnx-tf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 30.5 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.2
    Uninstalling typeguard-4.4.2:
      Successfully uninstalled typeguard-4.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
# Uninstall the existing tensorflow
!pip uninstall tensorflow -y


# Re-install onnx-tf to ensure compatibility with the new tensorflow version
!pip uninstall onnx-tf

!pip uninstall tensorflow-addons==0.20.0

Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Successfully uninstalled tensorflow-2.15.0
Found existing installation: onnx-tf 1.10.0
Uninstalling onnx-tf-1.10.0:
  Would remove:
    /usr/local/bin/onnx-tf
    /usr/local/lib/python3.11/dist-packages/onnx_tf-1.10.0.dist-info/*
    /usr/local/lib/python3.11/dist-packages/onnx_tf/*
    /usr/local/lib/python3.11/dist-packages/test/*
    /usr/local/lib/python3.11/dist-packages/third_party/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.11/dist-packages/third_party/LICENSE
    /usr/local/lib/python3.11/dist-packages/third_party/eigen/Eigen/Core
    /usr/local/lib/python3.11/dist-packages/third_party/eigen/Eigen/src/Core/ArithmeticSequence.h
    /usr/local/lib/python3.11/dist-packages/third_party/eigen/Eigen/src/Core/Array.h
    /usr/local/lib/python3.11/dist-packages/third_party/eigen/Eigen/src/Core/ArrayBase.h
    /usr/local/lib/python3.11/dist-packages/third_party/eigen/Eigen/src

In [ ]:
!pip install tensorflow

  Using cached keras-3.9.2-py3-none-any.whl.metadata (6.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 2.6 MB/s eta 0:00:00
Using cached keras-3.9.2-py3-none-any.whl (1.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 95.5 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency

In [ ]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("./student_model_distilled")

# Load model TensorFlow (bắt buộc model hỗ trợ TF, nếu không có TF model bạn phải convert từ PyTorch)
model = TFAutoModelForSequenceClassification.from_pretrained("./student_model_distilled", from_pt=True)


All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [ ]:
model.save_pretrained("./student_model_tf", saved_model=True)


In [ ]:
import tensorflow as tf

saved_model_dir = "./student_model_tf/saved_model/1"

# Khởi tạo converter từ thư mục SavedModel
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

# (Tùy chọn) Giảm dung lượng model bằng cách enable quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Chuyển đổi sang TFLite
tflite_model = converter.convert()

# Lưu model TFLite ra file
with open("student_model.tflite", "wb") as f:
    f.write(tflite_model)


In [ ]:
import os

model_path = "student_model.tflite"
model_size = os.path.getsize(model_path) / (1024 * 1024)  # Đổi sang MB
print(f"Kích thước mô hình TFLite: {model_size:.2f} MB")


Kích thước mô hình TFLite: 47.40 MB


**KIỂM TRA HOẠT ĐỘNG MÔ HÌNH TỐI ƯU**

In [ ]:
!pip uninstall numpy -y
!pip install tensorflow


Found existing installation: numpy 1.24.3
Uninstalling numpy-1.24.3:
  Successfully uninstalled numpy-1.24.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 43.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have tensorflow 2.19.0 which is incompatible.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.19.0 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.19.0 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.1.3 which is incompatible.


In [ ]:
!pip install numpy --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 111.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have tensorflow 2.19.0 which is incompatible.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.19.0 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.19.0 which is inco

In [ ]:
!pip uninstall tensorflow -y
!pip uninstall numpy -y
!pip install tensorflow

Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0
Found existing installation: numpy 2.2.5
Uninstalling numpy-2.2.5:
  Successfully uninstalled numpy-2.2.5
  Using cached numpy-2.1.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 2.8 MB/s eta 0:00:00
Using cached numpy-2.1.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have tensorflow 2.19.0 which is incompatible.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.19.0 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incom

In [ ]:
import pickle
import numpy as np
import tensorflow as tf

# Load dữ liệu đã tokenize
with open("tokenized_encoded_labeled_logs.pkl", "rb") as f:
    inputs = pickle.load(f)

input_ids = inputs["input_ids"].astype(np.int64)
attention_mask = inputs["attention_mask"].astype(np.int64)

# Load mô hình
interpreter = tf.lite.Interpreter(model_path="student_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Đặt dữ liệu đầu vào
interpreter.set_tensor(input_details[0]['index'], input_ids)
if len(input_details) > 1:
    interpreter.set_tensor(input_details[1]['index'], attention_mask)

interpreter.invoke()

# Lấy kết quả
output = interpreter.get_tensor(output_details[0]['index'])

print("Output logits:", output)
print("Dự đoán nhãn:", np.argmax(output, axis=1))


RuntimeError: Failed to import transformers.models.auto.tokenization_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
All ufuncs must have type `numpy.ufunc`. Received (<ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>)